In [ ]:
from cgi import parse_multipart
from flask import request
import requests
from bs4 import BeautifulSoup
import zipfile
import json
import urllib.request
import os
import xml.etree.ElementTree as ET
import pandas as pd
from korDB_manager import KoreaDB_manager
from database  import DataBase
import re
import time
import numpy as np

db = DataBase(host = 'localhost', db_name="KOR_DB", pwd = "ehfvkfdl123@")
korDB = KoreaDB_manager(host='localhost',db_name='KOR_DB',pwd='ehfvkfdl123@')

api_key = "8ba06a012b644df0819f3035d024f905c71e0165"



In [6]:
# Function list
def getTotalStock(corp_code,bsns_year, report_code):

    URL = "https://opendart.fss.or.kr/api/stockTotqySttus.json"
    params = {
        'crtfc_key': api_key,
        'corp_code': corp_code,
        'bsns_year': bsns_year,
        'reprt_code': report_code
    }

    res = requests.get(URL, params=params).json()

    if 'list' not in res: 
        return None
    return pd.DataFrame(res['list'])

def getFinState(corp_code,bsns_year, report_code='11011'):
    URL = 'https://opendart.fss.or.kr/api/'
    URL += 'fnlttMultiAcnt.json' if ',' in corp_code else 'fnlttSinglAcnt.json'

    params = {
        'crtfc_key': api_key,
        'corp_code': corp_code,
        'bsns_year':  bsns_year,   # 사업년도
        'reprt_code': report_code,  # "11011": 사업보고서
    }

    res = requests.get(URL, params=params).json()

    if 'list' not in res: 
        return None
    return pd.DataFrame(res['list'])

def getFinStateAll(corp_code,bsns_year, report_code='11011',fs_div='CFS'):
    URL = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
    """ 
    report_code: 
        - "11013": 1분기보고서
        - "11012": 반기보고서
        - "11014": 3분기보고서
        - "11011": 사업보고서
    fs_div:
        - CFS: 연결재무제표
        - OFS: 재무제표
    """
    params = {
        'crtfc_key': api_key,
        'corp_code': corp_code,
        'bsns_year':  bsns_year,   
        'reprt_code': report_code,  
        'fs_div': fs_div,           
    }

    res = requests.get(URL, params=params).json()

    if 'list' not in res: 
        return None
    return pd.DataFrame(res['list'])



def checkIsNum(string: str) -> bool:
    string = str(string)
    if re.match(r"\-?[0-9\.]+", string):
        return True
    else: 
        return False

def checkLen(df: pd.DataFrame) -> bool:
    return True if len(df) == 1 else False

def rmDupl(data: list) -> list:
    return list(set(data))

def doCheck(data, name="data"):

    if checkLen(data): 
        data = data[0]
    else: 
        print(f"[E] {name} len : {len(data)}")
        return None
    
    if checkIsNum(data): 
        data = int(data)
        print(f"[D] {name}: {data}")
    
    else: 
        print(f"[E] {name}: None")
    
        return None
    
    return data


def getStockNum(df):

    total_stocks =  df.loc[
        df['se'].isin(['합계'])
        ]['distb_stock_co'].to_numpy()[0].replace(',','')
    try:
        searchfor = ['보통주','의결권 있는']
        normal_stocks =  df.loc[
            df['se'].str.contains("|".join(searchfor))
            ]['distb_stock_co'].to_numpy()[0].replace(',','') # 보통주
    except:
        normal_stocks = None

    try:
        searchfor = ['우선주','의결권 없는']

        prior_stocks =  df.loc[
            df['se'].str.contains("|".join(searchfor))
            ]['distb_stock_co'].to_numpy()[0].replace(',','')
    except:
        prior_stocks = None

    total_stocks = int(total_stocks) if checkIsNum(total_stocks) else 0
    normal_stocks = int(normal_stocks) if checkIsNum(normal_stocks) else 0
    prior_stocks = int(prior_stocks) if checkIsNum(prior_stocks) else 0

    print(f"[D] 주식수(합계): {total_stocks}")
    print(f"[D] 보통주: {normal_stocks}")
    print(f"[D] 우선주: {prior_stocks}")

    if prior_stocks == None:
        prior_stocks = 0

    return total_stocks, normal_stocks, prior_stocks

def getEquity(df):
    data = df.loc[
        df['sj_div'].isin(['BS']) &
        (df['account_id'].isin(['ifrs-full_Equity']) |
        df['account_id'].isin(['ifrs_Equity'])),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 자본 len: {len(data)}")
        print(f"[I] serching other equity value step 1 ..")

        data = df.loc[
            df['sj_div'].isin(['BS']) &
            df['account_id'].isin(['ifrs-full_EquityAttributableToOwnersOfParent']),
            'thstrm_amount'
            ].to_numpy()

    # condition 2
    if not checkLen(data):
        print(f"[W] 자본 len: {len(data)}")
        print(f"[I] serching other equity value step 2 ..")

        searchfor = ['^([^가-힣]+)?자본([^부채].)*총계']

        data = df.loc[
            df['sj_div'].isin(['BS']) &
            df['account_nm'].str.contains("|".join(searchfor)),
            'thstrm_amount'
            ].to_numpy()


    return doCheck(data,'자본')

def getLiability(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_Liabilities'])|
        df['account_id'].isin(['ifrs_Liabilities']) ,
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 부채 len: {len(data)}")
        print(f"[I] serching other liability value step 1 ..")

        searchfor = ['^([^가-힣]+)?부채(.)*총계']

        data = df.loc[
            df['sj_div'].isin(['BS']) &
            df['account_nm'].str.contains("|".join(searchfor)),
            'thstrm_amount'
            ].to_numpy()

    return doCheck(data,'부채')




def getProfit(df):

    data = df.loc[
        df['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent'])|
        df['account_id'].isin(['ifrs_ProfitLossAttributableToOwnersOfParent']),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 당기순이익 len: {len(data)}")
        print(f"[I] serching other profit value step 1 ..")

        searchfor = ['^당기(.)*순이익','^연결당기순이익']

        index = df.loc[
            (df['account_nm'].str.contains('|'.join(searchfor)) |
            df['account_id'].isin(['ifrs-full_ProfitLoss'])) & 
            df['sj_div'].isin(['CIS']) 
            ].index.to_numpy()

        temp = df.loc[
            (df['account_nm'].str.contains('|'.join(searchfor)) |
            df['account_id'].isin(['ifrs-full_ProfitLoss'])) & 
            df['sj_div'].isin(['CIS']) 
            ]
        print(temp)
        if checkLen(index): 
            index = index[0]
            searchfor = ['지배기업']

            sub_df = df.loc[[index+1,index+2]]
            data = sub_df.loc[
                df['account_nm'].str.contains('|'.join(searchfor)) ,
                'thstrm_amount'
                ].to_numpy()
    # condition 2
    if not checkLen(data):
        print(f"[W] 당기순이익 len: {len(data)}")
        print(f"[I] serching other profit value step 2 ..")

        searchfor_nm = ['^([^가-힣]+)?당기(.)*순이익(\(손실\))?$','^([^가-힣]+)?연결당기순이익(\(손실\))?$']
        searchfor_id = ['^ifrs_ProfitLoss$', '^ifrs-full_ProfitLoss$']

        data = df.loc[
            (df['account_nm'].str.contains('|'.join(searchfor_nm)) |
            df['account_id'].str.contains('|'.join(searchfor_id))) & 
            df['sj_div'].isin(['CIS']) ,
            'thstrm_amount'
            ].to_numpy()
        temp = df.loc[
            (df['account_nm'].str.contains('|'.join(searchfor_nm)) |
            df['account_id'].str.contains('|'.join(searchfor_id))) & 
            df['sj_div'].isin(['CIS'])  
            ]      
        print(temp)  
        data = rmDupl(data)

    # condition 3
    if not checkLen(data):
        print(f"[W] 당기순이익 len: {len(data)}")
        print(f"[I] serching other profit value step 3 ..")

        searchfor_nm = ['^([^가-힣]+)?당기(.)*순이익(\(손실\))?$','^([^가-힣]+)?연결당기순이익(\(손실\))?$']
        searchfor_id = ['^dart_ProfitLossForStatementOfCashFlows$', ]

        data = df.loc[
            (df['account_nm'].str.contains('|'.join(searchfor_nm)) |
            df['account_id'].str.contains('|'.join(searchfor_id))) & 
            df['sj_div'].isin(['CF']) ,
            'thstrm_amount'
            ].to_numpy()
        temp = df.loc[
            (df['account_nm'].str.contains('|'.join(searchfor_nm)) |
            df['account_id'].str.contains('|'.join(searchfor_id))) & 
            df['sj_div'].isin(['CF'])  
            ]      
        print(temp)  
        data = rmDupl(data)



    return doCheck(data,'당기순이익')

def getRevenue(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_Revenue']) |
        df['account_id'].isin(['ifrs_Revenue']),
        'thstrm_amount'
        ].to_numpy()
    
    # condition 1
    if not checkLen(data):
        print(f"[W] 매출액 len: {len(data)}")
        print(f"[I] serching other revenue value step 1 ..")
        searchfor = ['^([^가-힣]+)?매출액']
        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)),
            'thstrm_amount'
            ].to_numpy()    

    # condition 2
    if not checkLen(data):
        print(f"[W] 매출액 len: {len(data)}")
        print(f"[I] serching other revenue value step 2 ..")
        searchfor = ['^([^가-힣]+)?영업수익']
        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)),
            'thstrm_amount'
            ].to_numpy()    

    return doCheck(data, '매출액')


def getGP(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_GrossProfit']) |
        df['account_id'].isin(['ifrs_GrossProfit']),
        'thstrm_amount'
        ].to_numpy()

    return doCheck(data, '매출총이익')


def getOpIncome(df):
    data = df.loc[
        df['account_id'].isin(['dart_OperatingIncomeLoss']),
        'thstrm_amount'
        ].to_numpy()

    if not checkLen(data):
        print(f"[W] opIncome len: {len(data)}")
        print(f"[I] serching other op_income value step 1 ..")
        searchfor = ['^([^가-힣]+)?영업이익$']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)),
            'thstrm_amount'
            ].to_numpy()    

    return doCheck(data,'영업이익')


def getCurrAssets(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_CurrentAssets']) | 
        df['account_id'].isin(['ifrs_CurrentAssets']),
        'thstrm_amount'
        ].to_numpy()

    return doCheck(data, "유동자산")


def getNonCurrAssets(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_NoncurrentAssets']) |
        df['account_id'].isin(['ifrs_NoncurrentAssets']) ,
        'thstrm_amount'
        ].to_numpy()
    
    return doCheck(data, "비유동부채")


def getCurrLiability(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_CurrentLiabilities']) |
        df['account_id'].isin(['ifrs_CurrentLiabilities']),
        'thstrm_amount'
        ].to_numpy()

    return doCheck(data, "유동부채")

def getCashAssets(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_CashAndCashEquivalents']) |
        df['account_id'].isin(['ifrs_CashAndCashEquivalents']),
        'thstrm_amount'
        ].to_numpy()

    return doCheck(data, "현금및현금성자산")


def getTaxCost(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_ProfitLossBeforeTax']) |
        df['account_id'].isin(['ifrs_ProfitLossBeforeTax']),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 법인세비용차감전순이익 len: {len(data)}")
        print(f"[I] serching other op_cash_flow value step 1 ..")
        searchfor = ['(.)*법인세비용차감전순이익']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)),
            'thstrm_amount'
            ].to_numpy()

    return doCheck(data,'법인세비용차감전순이익')


def getFinIncome(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_FinanceIncome']) |
        df['account_id'].isin(['ifrs_FinanceIncome']),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 금융수익 len: {len(data)}")
        print(f"[I] serching other capital value step 1 ..")
        searchfor = ['^([^가-힣]+)?금융수익$']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)),
            'thstrm_amount'
            ].to_numpy()

    return doCheck(data,'금융수익')


def getFinCost(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_FinanceCosts']) |
        df['account_id'].isin(['ifrs_FinanceCosts']),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 금융비용 len: {len(data)}")
        print(f"[I] serching other capital value step 1 ..")
        searchfor = ['^([^가-힣]+)?금융비용$']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)),
            'thstrm_amount'
            ].to_numpy()

    return doCheck(data,'금융비용')


def getCap(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_IssuedCapital']) |
        df['account_id'].isin(['ifrs_IssuedCapital']) ,
        'thstrm_amount'
        ].to_numpy()
    
    # condition 1
    if not checkLen(data):
        print(f"[W] 자본금 len: {len(data)}")
        print(f"[I] serching other capital value step 1 ..")
        searchfor = ['^([^가-힣]+)?자본금$']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)) &
            df['sj_div'].isin(['BS']) ,
            'thstrm_amount'
            ].to_numpy()
    
    # # condition 2
    # if not checkLen(data):
    #     print(f"[W] 자본금 len: {len(data)}")
    #     print(f"[I] serching other capital value step 2 ..")
    #     searchfor = ['자본금$']

    #     data = df.loc[
    #         df['account_nm'].str.contains('|'.join(searchfor)) &
    #         df['sj_div'].isin(['BS']) ,
    #         'thstrm_amount'
    #         ].to_numpy()


    return doCheck(data,'자본금')


def getOpCashFlow(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_CashFlowsFromUsedInOperatingActivities']) |
        df['account_id'].isin(['ifrs_CashFlowsFromUsedInOperatingActivities']),
        'thstrm_amount'
        ].to_numpy()
    
    # condition 1
    if not checkLen(data):
        print(f"[W] 영업활동현금흐름 len: {len(data)}")
        print(f"[I] serching other op_cash_flow value step 1 ..")
        searchfor = ['(.)*영업활동(.)*현금흐름$']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)) &
            df['sj_div'].isin(['CF']) ,
            'thstrm_amount'
            ].to_numpy()

    return doCheck(data,'영업활동현금흐름')

def getInvestCashFlow(df):
    data = df.loc[
        df['account_id'].isin(['ifrs-full_CashFlowsFromUsedInInvestingActivities']) |
        df['account_id'].isin(['ifrs_CashFlowsFromUsedInInvestingActivities']),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 투자활동현금흐름 len: {len(data)}")
        print(f"[I] serching other capital value step 1 ..")
        searchfor = ['(.)*투자활동(.)*현금흐름$']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)) &
            df['sj_div'].isin(['CF']) ,
            'thstrm_amount'
            ].to_numpy()


    return doCheck(data,'투자활동현금흐름')


def getFinCashFlow(df):

    data = df.loc[
        df['account_id'].isin(['ifrs-full_CashFlowsFromUsedInFinancingActivities']) |
        df['account_id'].isin(['ifrs_CashFlowsFromUsedInFinancingActivities']),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 재무활동현금흐름 len: {len(data)}")
        print(f"[I] serching other capital value step 1 ..")
        searchfor = ['(.)*재무활동(.)*현금흐름$']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)) &
            df['sj_div'].isin(['CF']) ,
            'thstrm_amount'
            ].to_numpy()

    
    return doCheck(data,'재무활동현금흐름')
        

def getCAPEX(df):


    data = df.loc[
        df['account_id'].isin(['ifrs-full_PurchaseOfPropertyPlantAndEquipmentClassifiedAsInvestingActivities']),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 유형자산의취득 len: {len(data)}")
        print(f"[I] serching other CAPEX step 1 ..")
        searchfor = ['유형자산(.)*취득']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)) &
            df['sj_div'].isin(['CF']) ,
            'thstrm_amount'
            ].to_numpy()

    return doCheck(data,'유형자산의취득')



def getFCF(df):
    data = df.loc[
        df['sj_div'].isin(['CF']) &
        df['account_id'].isin(['ifrs-full_PurchaseOfPropertyPlantAndEquipmentClassifiedAsInvestingActivities']),
        'thstrm_amount'
        ].to_numpy()
    
    return doCheck(data,'FCF')


def getCapSurpl(df):

    data = df.loc[
        df['account_id'].isin(['dart_CapitalSurplus']),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 자본잉여금 len: {len(data)}")
        print(f"[I] serching other capital surplus value step 1 ..")
        searchfor = ['^([^가-힣]+)?자본잉여금$']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)) ,
            'thstrm_amount'
            ].to_numpy()

    return doCheck(data,'자본잉여금')


def showFinState(state):
    for key,val in state.items():
        print(f"{key}:\t{val}")

def getIncomeSurpl(df):

    data = df.loc[
        df['account_id'].isin(['ifrs-full_RetainedEarnings']) |
        df['account_id'].isin(['ifrs_RetainedEarnings']),
        'thstrm_amount'
        ].to_numpy()

    # condition 1
    if not checkLen(data):
        print(f"[W] 이익잉여금 len: {len(data)}")
        print(f"[I] serching other income surplus value step 1 ..")
        searchfor = ['^([^가-힣]+)?이익잉여금$']

        data = df.loc[
            df['account_nm'].str.contains('|'.join(searchfor)) ,
            'thstrm_amount'
            ].to_numpy()


    return doCheck(data,'이익잉여금')



def prepFinState(stock_code, year):


    result = {}


    total_stocks, normal_stocks, prior_stocks = getStockNum(data_tot_stocks) # 총 발행 주식 수 
    equity = getEquity(data_fin_state) # 자본
    liability = getLiability(data_fin_state) # 부채
    assets = equity + liability # 자산
    profit = getProfit(data_fin_state) # 당기순이익
    revenue = getRevenue(data_fin_state) # 매출액
    # revenue = getRevenue(data_fin_state) # 매출액
    gp = getGP(data_fin_state) # 매출 총 이익
    op_income = getOpIncome(data_fin_state) # 영업이익
    curr_asset = getCurrAssets(data_fin_state) # 유동자산
    noncurr_asset = getNonCurrAssets(data_fin_state) # 비유동자산
    curr_liability =  getCurrLiability(data_fin_state) # 유동부채
    cash_asset = getCashAssets(data_fin_state) # 현금성 자산
    tax_cost = getTaxCost(data_fin_state) # 법인세 비용
    fin_income = getFinIncome(data_fin_state) # 금융수익
    fin_cost = getFinCost(data_fin_state) # 금융비용
    cap = getCap(data_fin_state) # 자본금
    opcash_flow = getOpCashFlow(data_fin_state) # 영업활동현금흐름
    investcash_flow = getInvestCashFlow(data_fin_state) # 투자활동현금흐름
    fincash_flow = getFinCashFlow(data_fin_state) # 재무활동현금흐름
    capex = getCAPEX(data_fin_state) # 유형자산의 취득
    cap_surpl = getCapSurpl(data_fin_state) # 자본 잉여금
    income_surpl = getIncomeSurpl(data_fin_state) # 이익 잉여금

    last_data = db.getDailyPrice(stock_code,start_date=f"{year}-12-20", end_date=f"{year}-12-31")

    if not last_data.empty:
        print(f"[D] last_data: {last_data}")

        last_stock_price = last_data.iloc[-1]['close']
        print(f"[D] last_stock_price: {last_stock_price}")
    else:
        print(f"[E] last_data is empty!")
        last_stock_price = None

    # EPS
    if checkIsNum(profit) & checkIsNum(total_stocks) :
        EPS = profit / total_stocks      
    else:
        print(f"[D] EPS --")
        print(f"profit: {checkIsNum(profit)}")
        print(f"total_stocks: {checkIsNum(total_stocks)}")
        EPS = None

    # PER
    if checkIsNum(last_stock_price) & checkIsNum(EPS) :
        PER = last_stock_price / EPS    # 시가총액 / 당기순이익
    else:
        print(f"[D] PER --")
        print(f"last_stock_price: {checkIsNum(last_stock_price)}")
        print(f"EPS: {checkIsNum(EPS)}")
        PER = None

    # BPS
    if checkIsNum(equity) & checkIsNum(total_stocks) :
        BPS = equity / total_stocks     # 자본총계 / 주식수 
    else:
        print(f"[D] BPS --")
        print(f"equity: {checkIsNum(equity)}")
        print(f"total_stocks: {checkIsNum(total_stocks)}")
        BPS = None

    # PBR
    if checkIsNum(last_stock_price) & checkIsNum(BPS) :
        PBR = last_stock_price / BPS    # 시가총액 / 자본 
    else:
        print(f"[D] PBR --")
        print(f"last_stock_price: {checkIsNum(last_stock_price)}")
        print(f"BPS: {checkIsNum(BPS)}")
        PBR = None

    # ROE
    if checkIsNum(PBR) & checkIsNum(PER) :
        ROE = PBR / PER * 100           # 당기순이익 / 자본
    else:
        print(f"[D] ROE --")
        print(f"PBR: {checkIsNum(PBR)}")
        print(f"PER: {checkIsNum(PER)}")
        ROE = None

    # ROA
    if checkIsNum(profit) & checkIsNum(assets) :
        ROA = profit / assets * 100     # 당기순이익 / 자산
    else:
        print(f"[D] ROA --")
        print(f"profit: {checkIsNum(profit)}")
        print(f"assets: {checkIsNum(assets)}")
        ROA = None

    # PSR
    if checkIsNum(last_stock_price) & checkIsNum(revenue) & checkIsNum(total_stocks):
        PSR = last_stock_price / (revenue / total_stocks) # 시가총액 /매출
    else:
        print(f"[D] PSR --")
        print(f"last_stock_price: {checkIsNum(last_stock_price)}")
        print(f"revenue: {checkIsNum(revenue)}")
        print(f"total_stocks: {checkIsNum(total_stocks)}")
        PSR = None

   # ROC
    if checkIsNum(op_income) & checkIsNum(assets) & checkIsNum(curr_liability):
        ROC = op_income / (assets-curr_liability) * 100
    else:
        print(f"[D] ROC --")
        print(f"op_income: {checkIsNum(op_income)}")
        print(f"assets: {checkIsNum(assets)}")
        print(f"curr_liability: {checkIsNum(curr_liability)}")
        ROC = None

   # FCF
    if checkIsNum(opcash_flow) & checkIsNum(capex):
        FCF = opcash_flow - capex
    else:
        print(f"[D] FCF --")
        print(f"opcash_flow: {checkIsNum(opcash_flow)}")
        print(f"capex: {checkIsNum(capex)}")
        FCF = None

    # EV_EBIT
    if checkIsNum(last_stock_price) & checkIsNum(total_stocks) & checkIsNum(liability) \
        & checkIsNum(cash_asset) & checkIsNum(fin_cost) &checkIsNum(tax_cost) & checkIsNum(fin_income) \
        & checkIsNum(profit):
        EBIT = profit + tax_cost - fin_income + fin_cost
        EV = last_stock_price * total_stocks + liability - cash_asset # 시가총액 + 부채 - (현금 + 비영업자산) 
        EV_EBIT= EV / EBIT
    else:
        print(f"[D] EV_EBIT --")
        print(f"cash_asset: {checkIsNum(cash_asset)}")
        print(f"fin_cost: {checkIsNum(fin_cost)}")
        print(f"tax_cost: {checkIsNum(tax_cost)}")
        print(f"fin_income: {checkIsNum(fin_income)}")
        EV_EBIT = None

    # GP/A
    if checkIsNum(gp) & checkIsNum(assets):
        GP_A = gp / assets * 100 
    else:
        print(f"[D] GP_A --")
        print(f"gp: {checkIsNum(gp)}")
        print(f"assets: {checkIsNum(assets)}")
        GP_A = None

    # 유보율
    if checkIsNum(cap_surpl) & checkIsNum(income_surpl) & checkIsNum(cap):
        reserve_ratio = (cap_surpl+income_surpl)/cap * 100
    else:
        print(f"[D] reserve_ratio --")
        print(f"cap_surpl: {checkIsNum(cap_surpl)}")
        print(f"income_surpl: {checkIsNum(income_surpl)}")
        print(f"cap: {checkIsNum(cap)}")
        reserve_ratio = None
    
    # 부채율
    if checkIsNum(liability) & checkIsNum(equity) :
        liability_ratio = liability / equity * 100
    else:
        print(f"[D] liability_ratio --")
        print(f"cap_surpl: {checkIsNum(cap_surpl)}")
        print(f"income_surpl: {checkIsNum(income_surpl)}")
        print(f"cap: {checkIsNum(cap)}")
        liability_ratio = None

    # 영업이익률
    if checkIsNum(op_income) & checkIsNum(revenue) :
        opInocome_ratio = op_income / revenue * 100
    else:
        print(f"[D] opInocome_ratio --")
        print(f"op_income: {checkIsNum(op_income)}")
        print(f"revenue: {checkIsNum(revenue)}")
        opInocome_ratio = None

    # 순이익률
    if checkIsNum(profit) & checkIsNum(revenue) :
        profit_ratio = profit / revenue *100
    else:
        print(f"[D] profit ratio --")
        print(f"op_income: {checkIsNum(op_income)}")
        print(f"revenue: {checkIsNum(revenue)}")
        profit_ratio = None

    result['연도'] = year
    result['종목코드'] = stock_code
    result['매출액'] = revenue
    result['매출총이익'] = gp
    result['영업이익'] = op_income
    result['당기순이익'] = profit
    result['자산총계'] = assets
    result['부채총계'] = liability
    result['자산총계'] = equity
    result['유동자산'] = curr_asset
    result['비유동자산'] = noncurr_asset
    result['유동부채'] = curr_liability
    result['자본금'] = cap
    result['영업활동현금흐름'] = opcash_flow
    result['투자활동현금흐름'] = investcash_flow
    result['재무활동현금흐름'] = fincash_flow
    result['CAPEX'] = capex
    result['FCF'] = FCF
    result['영업이익률'] = opInocome_ratio
    result['순이익률'] = profit_ratio
    result['EV_EBIT'] = EV_EBIT
    result['ROE'] = ROE
    result['ROA'] = ROA
    result['ROC'] = ROC
    result['부채비율'] = liability_ratio
    result['자본유보율'] = reserve_ratio
    result['EPS'] = EPS
    result['PER'] = PER
    result['BPS'] = BPS
    result['PSR'] = PSR
    result['PBR'] = PBR
    result['GP_A'] = GP_A
    result['발행주식수합계'] = total_stocks
    result['발행주식수보통주'] = normal_stocks
    result['발행주식수우선주'] = prior_stocks

    return result

        

In [ ]:
code_list = list(korDB.code_dict.keys())
print(f"[I] total stocks len: {len(code_list)}")
os.makedirs("fsdata/{YEAR}",exist_ok=True)

for i, code in enumerate(code_list[1860:]): # 880

    YEAR="2019"
    report_code = "11011"
    print(code)
    corpCode = korDB.stockCode2corpCode(code)

    data_tot_stocks = getTotalStock(corpCode, YEAR, report_code)
    data_fin_state = getFinStateAll(corpCode, YEAR, report_code)

    # makedir
    filename = f'fsdata/{YEAR}/{code}.csv'
    print(i,code)

    if (data_fin_state is not None) & (data_tot_stocks is not None): 

        data_fin_state.to_csv(filename,encoding='euc-kr')
        
        # preprocess
        data_fin_state.replace('',np.nan, inplace=True)
        data_fin_state.dropna(subset=['thstrm_amount'],inplace=True)
        data_fin_state = data_fin_state.reset_index(drop=True)
        result = prepFinState(code, YEAR)

        showFinState(result)
    else:
        print(f"[E] there is no date [code]:{code}")
        
    print("--------------------------")
    if i%60 == 30: 
        print(f"[I] sleep 10 sec ..")
        time.sleep(10)



In [7]:
YEAR="2019"
code = "208340" 
report_code = "11011"

corpCode = korDB.stockCode2corpCode(code)

data_tot_stocks = getTotalStock(corpCode, YEAR, report_code)
data_fin_state = getFinStateAll(corpCode, YEAR, report_code)

# preprocess
data_fin_state.replace('',np.nan, inplace=True)
data_fin_state.dropna(subset=['thstrm_amount'],inplace=True)
data_fin_state = data_fin_state.reset_index(drop=True)


# make dir
os.makedirs(f"fsdata/{YEAR}",exist_ok=True)
filename = f'fsdata/{YEAR}/{code}.csv'
print(f"code: {code}")
if (data_fin_state is not None) & (data_tot_stocks is not None): 
    data_fin_state.to_csv(filename,encoding='euc-kr')

    result = prepFinState(code, YEAR)

    showFinState(result)
else:
    print(f"[E] there is no date [code]:{code}")
    
print("--------------------------")


code: 208340
[D] 주식수(합계): 6910809
[D] 보통주: 6910809
[D] 우선주: 0
[D] 자본: 67783266776
[D] 부채: 93563709415
[D] 당기순이익: -10158298177
[W] 매출액 len: 0
[I] serching other revenue value step 1 ..


/home/coolseaweed/ENV/miniconda3/envs/stone82/lib/python3.6/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


[W] 매출액 len: 0
[I] serching other revenue value step 2 ..
[E] 매출액 len : 0
[E] 매출총이익 len : 0
[D] 영업이익: -10104255493
[D] 유동자산: 136161516484
[D] 비유동부채: 25185459707
[D] 유동부채: 2623515725
[D] 현금및현금성자산: 46217279816
[D] 법인세비용차감전순이익: -10158298177
[D] 금융수익: 9752022259
[D] 금융비용: 8904734354
[D] 자본금: 3455404500
[D] 영업활동현금흐름: -7230239735
[D] 투자활동현금흐름: -74167617247
[D] 재무활동현금흐름: 97131083009
[W] 유형자산의취득 len: 0
[I] serching other CAPEX step 1 ..
[E] 유형자산의취득 len : 0
[D] 자본잉여금: 111422029201
[D] 이익잉여금: -47117748031
[D] last_data:              open   high    low  close  diff  volume
date                                                
2019-12-20  37000  37000  36000  36100   900   59952
2019-12-23  36150  36750  35650  36100     0   49240
2019-12-24  36100  36250  35350  36100     0   37773
2019-12-26  35900  36300  35250  35750   350   66657
2019-12-27  36000  37200  36000  37100  1350   98237
2019-12-30  37050  39100  37050  38650  1550  168533
[D] last_stock_price: 38650
last_stock_price: True
revenue: 

In [ ]:
code = "092190"
update_df = korDB.crawlDataFromNaver(code=code)
korDB.replaceIntoDB(update_df,1,code,'temp')

In [ ]:
last_data = db.getDailyPrice("039310",start_date=f"2018-12-29", end_date=f"{YEAR}-12-31")
print(f"[D] last_data: {last_data}")

In [ ]:


stockCode ="00040" #004690
# stockCode ="005930"
YEAR="2019"
report_code = "11011"

data_tot_stocks = getTotalStock(corpCode, YEAR, report_code)
data_fin_state = getFinStateAll(corpCode, YEAR, report_code)
os.makedirs("fsdata",exist_ok=True)
filename = f'fsdata/{stockCode}_{YEAR}.csv'
data_fin_state.to_csv(filename,encoding='euc-kr')
data_tot_stocks